In [1]:
from collections import namedtuple
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from Deep_Classifier.constants import *
from Deep_Classifier.utils import read_yaml, create_directories

In [4]:
class Configuration:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH,
                parameters_file_path = PARAMETERS_FILE_PATH
                ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(parameters_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_checkpoint),
            Path(config.tensorboard_root_log_dir)
            ]
        )
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tensorboard_callbacks(self):
        time_stamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_running_logdir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tensorboard_logs_at{time_stamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tensorboard_running_logdir)
    
    @property
    def _create_checkpoint_callbacks(self):

        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tensorboard_checkpoint_callbacks(self):
        return [
            self._create_tensorboard_callbacks,
            self._create_checkpoint_callbacks
        ]

In [6]:
try:
    config = Configuration()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    prepare_callbacks.get_tensorboard_checkpoint_callbacks()

except Exception as e:
    raise e